# 네이버 리뷰 데이터를 활용한 한국어 감성분석
네이버 영화 리뷰데이터(Naver Sentiment Movie Corpus,NSMC)를 활용


In [2]:
import os

import numpy as np
import pandas as pd

from datetime import datetime
import json
import re

from konlpy.tag import Okt # komoran, han, kkma

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
import tensorflow as tf

from tqdm.notebook import tqdm

## 데이터 불러오기

In [3]:
train = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', header=0, delimiter='\t' ,quoting=3)
test = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', header=0, delimiter='\t' ,quoting=3)

In [4]:
display(train.head())
display(test.head())

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [6]:
train.shape, test.shape

((150000, 3), (50000, 3))

In [7]:
# 정규표현식 연습
# 외부적 특징 먼저 생각, 그 다음 내부적 특징을 생각해야 실수를 줄일 수 있다.
# 외부적 특징으로 표현하고 부족하면 내부적 특징을 추가.
name_info = '서연 1 10,353 서윤 2 10,001 '

# 내부적 특징 표현
re.findall('\d+,+\d+', name_info)

# 외부적 특징 표현
# 외부적 특징으로 표현할때는 앞/뒤 모두 표현 가능해야 한다.
re.findall('[가-힣]+\s[0-9]+\s(.+?)\s', name_info)
### 이렇게 쓸 때 Quantifier(.+?) 사용!
# '한글+\s숫자+\s()\s' 사이에 있는거 다 찾아와!

['10,353', '10,001']

## 데이터 전처리

### Implementation 1

In [8]:
def preprocessing(review, okt, remove_stopwords = False, stop_words = []):
    # 함수의 인자는 다음과 같다.
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 생성하지 않고 미리 생성후 인자로 받는다.
    # remove_stopword : 불용어를 제거할지 선택 기본값은 False
    # stop_word : 불용어 사전은 사용자가 직접 입력해야함 기본값은 비어있는 리스트

    # 1. 한글 및 공백을 제외한 문자 모두 제거. 정규표현식 사용
    if type(review) != str:
        return []
    word_review = re.sub('[^ㄱ-ㅎ가-힣 ]', '', review)

    # 2. okt 객체를 활용해서 형태소 단위(품사까지 원형으로 반환)로 나눈다.
    # token = okt.morphs(review)
    word_review = okt.pos(word_review, stem = True)
    # 3. 불용어 제거
    # word_review = [t for t in word_review if t[0] not in stop_words]

    # 4. 노이즈 제거 (글자 1개를 노이즈로 간주하고 제거하도록 하자.)
    # word_review = [t for t in word_review if len(t[0]) > 1]

    # 5. 명사, 동사, 형용사만 추출
    # word_review = [t[0] for t in word_review if t[1] in ['Noun', 'Verb', 'Adjective']]
    
    # 3 + 4 + 5
    word_review = [token for token, pos in word_review if (token not in stop_words) & (len(token) > 1) & (pos in ['Noun', 'Verb', 'Adjective'])]

    return word_review

In [9]:
sample_review = train['document'][0]
sample_review

'아 더빙.. 진짜 짜증나네요 목소리'

In [10]:
# stop_words = [ '은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한']
stop_words = ['목소리']

from konlpy.tag import Okt # komoran, han, kkma
okt = Okt()
preprocessing(sample_review, okt, remove_stopwords = True, stop_words=stop_words)

['더빙', '진짜', '짜증나다']

In [11]:
# stop_words = [ '은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한']
stop_words = ['목소리']
okt = Okt()
clean_review = []
clean_review_test = []


# for review in tqdm(train['document']):
#     # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
#     if type(review) == str:
#         clean_review.append(preprocessing(review, okt, remove_stopwords = True, stop_words=stop_words))
#     else:
#         clean_review.append([])
# for review in tqdm(test['document']):
#     # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
#     if type(review) == str:
#         clean_review_test.append(preprocessing(review, okt, remove_stopwords = True, stop_words=stop_words))
#     else:
#         clean_review_test.append([])

# for문을 apply로 바꿔보기

clean_review = train['document'].apply(preprocessing, args = (okt, True, stop_words))
clean_review_test = test['document'].apply(preprocessing, args = (okt, True, stop_words))

In [38]:
# pickle 파일로 저장하기
import pickle

with open('./clean_revie.pickle', 'wb') as f:
  pickle.dump(clean_review, f)
  
with open('./clean_review_test.pickle', 'wb') as f:
  pickle.dump(clean_review_test, f)

In [ ]:
# pickle 파일 불러오기
# import pickle

# with open('./clean_review.pickle', 'rb') as f:
#   clean_review = pickle.load(f)

# with open('./clean_review_test.pickle', 'rb') as f:
#   clean_review_test = pickle.load(f)

In [39]:
print(len(clean_review))
print(len(clean_review_test))

150000
50000


In [12]:
tokenizer = Tokenizer()
# scaling fit처럼 하는 fit. 토큰-숫자 mapping table 만들어 준다.
tokenizer.fit_on_texts(clean_review) # 단어 인덱스 구축
# mapping
text_sequences = tokenizer.texts_to_sequences(clean_review) # 문자열 -> 인덱스 리스트
                                                            # '나는 천재다 나는 멋있다' -> [1, 2, 1, 3]
# train data로 fit 시켜준 tokenizer로 test data도 인덱스로 변경
# train에 없던 단어가 test에 있으면? OutOfVocabulary 문제. 최소화는 다음주에 배울 것.(sub of word tokenizer?)
text_sequences_test = tokenizer.texts_to_sequences(clean_review_test)

word_vocab = tokenizer.word_index # 딕셔너리 형태
print("전체 단어 개수: ", len(word_vocab)) # 전체 단어 개수 확인

전체 단어 개수:  40498


In [13]:
# 문장별로 토큰 갯수가 다르다.
# padding 넣어서 토큰 갯수 동일하게 맞춰줘야 한다.
# post에 줄 수도 있고 pre에 줄 수도 있는데, 일반적으로는 pre에 준다. recurrent 돌다가 까먹는다.

MAX_SEQUENCE_LENGTH = 30 # 문장 최대 길이

X_train = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 50 단어가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_train = np.array(train['label']) # 각 리뷰의 감정을 넘파이 배열로 만든다.

print('Shape of input data tensor:', X_train.shape) # 리뷰 데이터의 형태 확인
print('Shape of label tensor:', y_train.shape) # 감정 데이터 형태 확인

Shape of input data tensor: (150000, 30)
Shape of label tensor: (150000,)


In [14]:
MAX_SEQUENCE_LENGTH = 30 # 문장 최대 길이
X_test = pad_sequences(text_sequences_test, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 50 단어가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_test = np.array(test['label']) # 각 리뷰의 감정을 넘파이 배열로 만든다.

print('Shape of input data tensor:', X_test.shape) # 리뷰 데이터의 형태 확인
print('Shape of label tensor:', y_test.shape) # 감정 데이터 형태 확인

Shape of input data tensor: (50000, 30)
Shape of label tensor: (50000,)


In [15]:
X_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0, 289,  11, 152])

## 모델 구축

### Implementation 2

In [88]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

model = Sequential()
# word embedding. Bof, TF-IDF 안쓴다. 
# 단어 하나씩 벡터화 해야 한다.
# 여기서 사용되는 Embedding은 Word2Vec처럼 Fancy한 애는 아니고 그냥 Naive한 아이.
model.add(Embedding(len(word_vocab)+1, 100)) # (단어집합의 크기, 임베딩 후 벡터 크기)
# => shape = (15만, 30, 100)

# Word2Vec, Embedding 된 Vectors.
# 벡터 크기가 100이라는거는 단어들을 100차원 공간에 우겨넣는것.
# 4만개가 100차원에 들어가려면 원핫인코딩으로는 안됨.
# 그래서 각각의 단어들이 1이 아닌 float 값을 가지게 된다.
# ==> 의미가 있는 단어들은 근처에 있도록 만들자. Queen - woman + man = King.

## lstm layer 구축
model.add(LSTM(64, return_sequences = True))
model.add(LSTM(30, return_sequences = True))
model.add(LSTM(15))
model.add(Dense(3))
model.add(Dense(1))
## output layer 구축(Dense), output = 긍정/부정

# callback도 넣어보자.

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics=['accuracy'])
# 이진 분류이므로 손실함수는 binary_crossentropy 사용, 에폭마다 정확도를 보기 위해 accuracy 적용
print(model.summary()) #모델 아키텍처 출력

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 100)         4049900   
                                                                 
 lstm_6 (LSTM)               (None, None, 64)          42240     
                                                                 
 lstm_7 (LSTM)               (None, None, 30)          11400     
                                                                 
 lstm_8 (LSTM)               (None, 15)                2760      
                                                                 
 dense_1 (Dense)             (None, 3)                 48        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 4,106,352
Trainable params: 4,106,352
No

## 모델 학습

### Implementation 3

In [89]:
## model fit
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train, y_train, epochs= 100, batch_size = 100, validation_split=0.2, callbacks=[early])

Epoch 1/100
1200/1200 [==============================] - 52s 38ms/step - loss: 0.5206 - accuracy: 0.7784 - val_loss: 0.4638 - val_accuracy: 0.8034
Epoch 2/100
1200/1200 [==============================] - 44s 37ms/step - loss: 0.4160 - accuracy: 0.8263 - val_loss: 0.4074 - val_accuracy: 0.8252
Epoch 3/100
1200/1200 [==============================] - 44s 37ms/step - loss: 0.3810 - accuracy: 0.8438 - val_loss: 0.3960 - val_accuracy: 0.8286
Epoch 4/100
1200/1200 [==============================] - 44s 37ms/step - loss: 0.3552 - accuracy: 0.8570 - val_loss: 0.3870 - val_accuracy: 0.8328
Epoch 5/100
1200/1200 [==============================] - 44s 37ms/step - loss: 0.3358 - accuracy: 0.8684 - val_loss: 0.3905 - val_accuracy: 0.8332
Epoch 6/100
1200/1200 [==============================] - 44s 37ms/step - loss: 0.3138 - accuracy: 0.8779 - val_loss: 0.4219 - val_accuracy: 0.8315
Epoch 7/100
1200/1200 [==============================] - 50s 41ms/step - loss: 0.3044 - accuracy: 0.8846 - val_loss: 0

In [80]:
X_train.shape

(150000, 30)

## 모델 검증

In [ ]:
from sklearn.metrics import accuracy_score

y_train_predclass = model.predict(X_train)
y_test_predclass = model.predict(X_test)

y_train_prediction = np.where(y_train_predclass > 0.5, 1, 0)
y_test_prediction = np.where(y_test_predclass > 0.5, 1, 0)

print("Train Accuracy: {}".format(round(accuracy_score(y_train, y_train_prediction),3)))
print("Test Accuracy: {}".format(round(accuracy_score(y_test, y_test_prediction),3)))

Train Accuracy: 0.91
Test Accuracy: 0.85


## 네이버웹툰 댓글에 감성라벨링하기

In [5]:
!gdown https://drive.google.com/uc?id=1s4w81tICR74bXPXGww6DNA6Xs6WtOZeX

'gdown'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [ ]:
import pandas as pd
# df_comment = pd.read_csv("네이버웹툰_댓글스크랩핑결과.csv")

train = pd.read_csv(r'data/train_final.csv')
test = pd.read_csv(r'data/test_final.csv')
df_comment = train

,contents,anonymous,best,country,lang,modTime,replyCount,sympathyCount,antipathyCount
0,역류성 식도염 치료비용 25만원 치료기간 3개월 깨어나세요 작가님,False,True,KR,ko,2022-02-19T23:03:56+0900,43,10989,22
1,나 이거 많이본듯 세상에 이런일이였나 동물농장이었나\n새끼소가 태어날때부터 약해서 ...,False,True,KR,ko,2022-02-19T23:10:10+0900,38,10518,16
2,폰이... 팩폭을?,False,True,KR,ko,2022-02-19T23:14:46+0900,41,9106,25
3,현재 새벽4시42분. 저는지금 출근중인 시내버스 기사입니다. 자까님 안녕히주무세요.,False,True,KR,ko,2022-02-21T04:42:53+0900,58,7597,12
4,이것은 '정각실행 강박 증후군'(juengkake joha haha)으로\n이 질병...,False,True,KR,ko,2022-02-19T23:12:51+0900,30,5608,33
...,...,...,...,...,...,...,...,...,...
330,독립일기만한 웹툰이 엄써요...데헷^^♥,False,True,KR,ko,2022-02-20T00:19:51+0900,2,5,1
331,ㅋㅋㅋㅋ ㅋㅋㅋㅋ 나랑 똑같애,False,True,KR,ko,2022-02-19T23:34:48+0900,0,5,0
332,진짜 등베개 광고 받으셔야해,False,True,KR,ko,2022-02-19T23:19:27+0900,0,5,0
333,지금이 그나마 겨울이라 괜찮긴 한데 자취생이 한여름에 설거지를 안하고 외출을 할 경...,False,True,KR,ko,2022-02-19T23:05:36+0900,0,5,0


### 웹툰 댓글 직접 크롤링
https://colab.research.google.com/drive/1nBJiK5UCcwHwwNipKi1-Z53gDeRGmQMv#scrollTo=28U1Ah7sGwBv

Json vs Dict
- Json은 문자열, Dict는 자료형
- Dict > Json : DumPs
- Json > Dict : loads

In [27]:
import requests
from bs4 import BeautifulSoup
import json

In [23]:
url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_cv=20230829152227&_callback=jQuery363021554426825589923_1693788810122&lang=ko&country=KR&objectId=814566_2&categoryId=&pageSize=15&indexSize=10&groupId=814566&listType=OBJECT&pageType=more&page=2&currentPage=1&refresh=false&sort=NEW&current=458688474&prev=458696844&moreParam.direction=next&moreParam.prev=063eq13p5bsxw&moreParam.next=063eelncve8aw&_=1693788810126'
headers = {
    'Referer':'https://comic.naver.com/webtoon/detail?titleId=814566&no=2',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
}
resp = requests.get(url, headers = headers)
resp

<Response [200]>

In [25]:
print(resp.text)
# HTTP Status 403 : Header 추가 필요

jQuery363021554426825589923_1693788810122({"success":true,"code":"1000","message":"요청을 성공적으로 처리하였습니다.","lang":"ko","country":"KR","result":{"commentList":[{"ticket":"comic","objectId":"814566_2","categoryId":"*","templateId":"webtoon","commentNo":"458688286","parentCommentNo":"458688286","replyLevel":1,"replyCount":0,"replyAllCount":0,"replyPreviewNo":null,"replyList":null,"imageCount":0,"imageList":null,"imagePathList":null,"imageWidthList":null,"imageHeightList":null,"commentType":"txt","stickerId":null,"sticker":null,"sortValue":1693755899869,"contents":"응원합니다 작가님","userIdNo":"3i4zB","exposedUserIp":null,"lang":"ko","country":"KR","idType":"naver","idProvider":"naver","userName":"시리우스","userProfileImage":null,"profileType":"comic","modTime":"2023-09-04T00:45:00+0900","modTimeGmt":"2023-09-03T15:45:00+0000","regTime":"2023-09-04T00:45:00+0900","regTimeGmt":"2023-09-03T15:45:00+0000","sympathyCount":1,"antipathyCount":0,"hideReplyButton":false,"status":0,"mine":false,"best":false,"men

In [78]:
# 댓글은 result commentList [] contents
# json.loads(resp.text[41:-2])
url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?'
params = {'ticket' : 'comic',
           'templateId' : 'webtoon',
           'pool' : 'cbox3',
           '_cv' : '20230829152227',
           'lang':'ko',
           'country':'KR',
           'objectId':'783053_94',
           'pageSize':'15',
           'indexSize':'10',
           'groupId':'783053',
           'listType':'OBJECT',
           'pageType':'more',
           'sort':'new',
           'page': '1'}
headers = {
    'Referer':'https://comic.naver.com/webtoon/detail?titleId=814566&no=2',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
}
dic_full = {}
for i in range(1, 95):
    params['objectId'] = '783053_' + str(i)
    dic_full[str(i)] = []
    n = 1
    while 1:
        params['page'] = str(n)
        resp = requests.get(url + str(n), headers = headers, params = params)
        n += 1
        resp_dic = json.loads(resp.text[resp.text.find('(') + 1:-2])

        commentList = resp_dic['result']['commentList']
        if len(commentList) == 0:
            break

        for comment in commentList:
            dic_full[str(i)].append(comment['contents'])
    print(f'{i} 완료')

1 완료
2 완료
3 완료
4 완료
5 완료
6 완료
7 완료
8 완료
9 완료
10 완료
11 완료
12 완료
13 완료
14 완료
15 완료
16 완료
17 완료
18 완료
19 완료
20 완료
21 완료
22 완료
23 완료
24 완료
25 완료
26 완료
27 완료
28 완료
29 완료
30 완료
31 완료
32 완료
33 완료
34 완료
35 완료
36 완료
37 완료
38 완료
39 완료
40 완료
41 완료
42 완료
43 완료
44 완료
45 완료
46 완료
47 완료
48 완료
49 완료
50 완료
51 완료
52 완료
53 완료
54 완료
55 완료
56 완료
57 완료
58 완료
59 완료
60 완료
61 완료
62 완료
63 완료
64 완료
65 완료
66 완료
67 완료
68 완료
69 완료
70 완료
71 완료
72 완료
73 완료
74 완료
75 완료
76 완료
77 완료
78 완료
79 완료
80 완료
81 완료
82 완료
83 완료
84 완료
85 완료
86 완료
87 완료
88 완료
89 완료
90 완료
91 완료
92 완료
93 완료
94 완료


In [95]:
li = []
for p in dic_full.values():
    li += p

In [94]:
len(li)

0

### Implementation 4

- preprocessing 함수 이용하여 전처리
- text_to_sequences()함수 이용하여 벡터화
- pad_sequence()함수 이용하여 벡터 크기 맞추기

In [96]:
## preprocessing 함수 이용하여 전처리


okt = Okt()
df_comment = pd.DataFrame(li, columns = ['document'])
clean_review = df_comment['document'].apply(preprocessing, args = (okt, True, stop_words))


## text_to_sequences()함수 이용하여 벡터화

text_sequences_webtoon = tokenizer.texts_to_sequences(clean_review)


## pad_sequence()함수 이용하여 벡터 크기 맞추기
new_text = pad_sequences(text_sequences_webtoon, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 50 단어가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.




In [97]:
pred = model.predict(new_text)
pred_label = np.where(pred > 0.5, 1, 0)
pred_label

3328/3328 [==============================] - 35s 10ms/step


array([[1],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [99]:
df_comment['pred_label'] = pred_label
df_comment[['document', 'pred_label']].tail(10)

,document,pred_label
106469,김준구 안 나와서 허전하면 개추 ㅋㅋ,0
106470,인격패치 0% 성한수 보고 싶은 사람 개추,0
106471,부러쓰 부러쓰 또 쿠키 구워 부러쓰,0
106472,쿠로다 유키에같은 닌자들이 나오네,0
106473,배댓 한번만 시켜줘,1
106474,개지린다,1
106475,ㅋㅋㅋㅋ,1
106476,ㅋㅋㅋㅋㅋ 개츄,1
106477,3,1
106478,2,1


In [102]:
df_comment.to_csv('labeled.csv')

출처 : https://github.com/reniew/NSMC_Sentimental-Analysis/blob/master/notebook/NSMC_Preprocessing.ipynb  